# Part 3. language model
Here will create Language model   
https://huggingface.co/blog/wav2vec2-with-ngram

In [1]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /home/talha/.huggingface/token


In [2]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-08-13 17:24:41--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K   320KB/s    in 1.5s    

2022-08-13 17:24:44 (320 KB/s) - written to stdout [491888/491888]



In [3]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZi

[ 68%] Building CXX object lm/builder/CMakeFiles/kenlm_builder.dir/corpus_count.cc.o
[ 69%] Building CXX object lm/builder/CMakeFiles/kenlm_builder.dir/initial_probabilities.cc.o
[ 70%] Building CXX object lm/builder/CMakeFiles/kenlm_builder.dir/interpolate.cc.o
[ 71%] Building CXX object lm/builder/CMakeFiles/kenlm_builder.dir/output.cc.o
[ 72%] Linking CXX executable ../bin/kenlm_benchmark
[ 72%] Built target kenlm_benchmark
Scanning dependencies of target phrase_table_vocab
[ 73%] Building CXX object lm/filter/CMakeFiles/phrase_table_vocab.dir/phrase_table_vocab_main.cc.o
[ 75%] Linking CXX executable ../../bin/phrase_table_vocab
[ 75%] Built target phrase_table_vocab
Scanning dependencies of target filter
[ 76%] Building CXX object lm/filter/CMakeFiles/filter.dir/filter_main.cc.o
[ 77%] Building CXX object lm/builder/CMakeFiles/kenlm_builder.dir/pipeline.cc.o
[ 78%] Linking CXX static library ../../lib/libkenlm_builder.a
[ 78%] Built target kenlm_builder
Scanning dependencies of ta

In [4]:
!ls

'1. clean-data.ipynb'	    data		    train-final-digits.ipynb
'3. language model.ipynb'   data-prep-final.ipynb   train-final.ipynb
'4. Final test .ipynb'	    kenlm		    Untitled.ipynb
 cache			    text4lm.csv		    urdumodel
 csv			    tex-test.csv	    vast-colab-trainer.ipynb


In [7]:
import pandas as pd
df=pd.read_csv('text4lm.csv')
df.dropna(inplace=True)
df.head()


,text
0,جی جناب یہ سمجھ لینے کے بعد کہ بزنس پلان کیا ہ...
1,اب ہم یہ سمجھیں گے یا اپنے اگلے دو سوالوں کے ج...
2,اس کو آپ پریکٹیکلی کیسے بزنس میں ٹرانسفارم کرو...
3,اور وہ بزنس پلان اصل میں آپ کو ہیلپ کر دے گا ک...
4,ہم اگلی چیز کی طرف اگلی بات کی طرف چلتے ہیں


In [8]:
with open("text.txt", "w") as file:
  file.write(" ".join(df.text.values))

In [9]:
!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /media/talha/New Volume/fiverr/urdu/data/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 683007 types 25224
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:302688 2:2622257408 3:4916732928 4:7866771968 5:11472376832
Statistics:
1 25223 D1=0.629061 D2=1.00345 D3+=1.34952
2 228086 D1=0.767622 D2=1.08521 D3+=1.46505
3 462646 D1=0.880522 D2=1.25999 D3+=1.46328
4 572404 D1=0.947022 D2=1.40804 D3+=1.58197
5 610227 D1=0.91461 D2=1.70154 D3+=1.74853
Memory estimate for binary LM:
type       kB
probing 40971 assuming -p 1.5
probing 48471 assuming -r models -p 1.5
trie    19100 without quantization
trie    10147 assuming -q 8 -b 8 quantization 
trie    17170 assuming -a 22 array pointer compression
trie     8217 assuming -a 22 -q 8 -b 8 array pointer com

In [10]:
!head -20 5gram.arpa

\data\
ngram 1=25223
ngram 2=228086
ngram 3=462646
ngram 4=572404
ngram 5=610227

\1-grams:
-5.376166	<unk>	0
0	<s>	-0.114852585
-3.1260948	جی	-0.34530106
-4.2932763	جناب	-0.17453776
-2.4780803	یہ	-0.42037845
-3.4255755	سمجھ	-0.45162588
-3.4626222	لینے	-0.5037698
-1.6532955	کے	-0.6494303
-3.4853032	بعد	-0.18375047
-2.6374352	کہ	-0.400029
-3.4853032	بزنس	-0.38422248
-3.933058	پلان	-0.26846772


In [11]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [12]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained('urdumodel',local_files_only=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(),
                                                     key=lambda item: item[1])}
print(len(sorted_vocab_dict))
sorted_vocab_dict

55


{'2': 0,
 '4': 1,
 'ج': 2,
 'ث': 3,
 'ز': 4,
 'ب': 5,
 'ص': 6,
 'ہ': 7,
 '8': 8,
 'چ': 9,
 'ژ': 10,
 '5': 11,
 '3': 12,
 'د': 13,
 'خ': 14,
 '7': 15,
 '9': 16,
 'ں': 17,
 'غ': 18,
 'ی': 19,
 'گ': 20,
 'ذ': 21,
 'ء': 22,
 'ح': 23,
 'آ': 24,
 'ض': 25,
 '1': 26,
 'ت': 27,
 'ل': 28,
 'پ': 29,
 'م': 30,
 'ک': 31,
 '0': 32,
 'س': 33,
 'ھ': 34,
 'ٹ': 35,
 'ڈ': 36,
 'ط': 37,
 'ن': 38,
 'ظ': 39,
 'ع': 40,
 'ا': 41,
 'ق': 42,
 '|': 43,
 '6': 44,
 'و': 45,
 'ے': 46,
 'ش': 47,
 'ڑ': 48,
 'ر': 49,
 'ف': 50,
 '[unk]': 51,
 '[pad]': 52,
 '<s>': 53,
 '</s>': 54}

In [16]:
#pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /media/talha/New Volume/fiverr/urdu/data/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


In [17]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

download the model locally 
https://discuss.huggingface.co/t/download-models-for-local-loading/1963

In [19]:
!ls 

'1. clean-data.ipynb'	    csv			    text.txt
'3. language model.ipynb'   data		    train-final-digits.ipynb
'4. Final test .ipynb'	    data-prep-final.ipynb   train-final.ipynb
 5gram.arpa		    kenlm		    Untitled.ipynb
 5gram_correct.arpa	    text4lm.csv		    urdumodel
 cache			    tex-test.csv	    vast-colab-trainer.ipynb


In [18]:
from huggingface_hub import Repository
repo = Repository(local_dir="urdumodel")


In [20]:
processor_with_lm.save_pretrained("urdumodel")

In [21]:
!kenlm/build/bin/build_binary urdumodel/language_model/5gram_correct.arpa urdumodel/language_model/5gram.bin

Reading urdumodel/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [22]:
!rm urdumodel/language_model/5gram_correct.arpa && tree -h urdumodel/

urdumodel/
├── [  30]  added_tokens.json
├── [ 513]  alphabet.json
├── [2.3K]  config.json
├── [ 368]  language_model
│   ├── [ 40M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [286K]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [2.7K]  README.md
├── [ 406]  special_tokens_map.json
├── [ 381]  tokenizer_config.json
├── [3.2K]  training_args.bin
└── [ 624]  vocab.json

1 directory, 13 files


In [ ]:
repo.push_to_hub('URDU-ASR-LM')